In [1]:
TODAY="2021-11-13"
TASK = "phylogenetic-determination"
DIR = "$(homedir())/$(TODAY)-$(TASK)"
if !isdir(DIR)
    mkdir(DIR)
end
cd(DIR)

In [24]:
import Pkg

pkgs = [
"JSON",
"HTTP",
"Dates",
"uCSV",
"DelimitedFiles",
"DataFrames",
"ProgressMeter",
"BioSequences",
"FASTX",
"Distances",
"Plots",
"StatsPlots",
"StatsBase",
"Statistics",
"Mmap",
"MultivariateStats",
"PyCall",
"Random",
"Primes",
"Revise",
"SparseArrays",
"SHA",
"Mycelia",
"GenomicAnnotations",
"BioFetch",
"Combinatorics",
"StaticArrays",
"BioSymbols"
]

for pkg in pkgs
    try
        eval(Meta.parse("import $pkg"))
    catch
        Pkg.add(pkg)
        eval(Meta.parse("import $pkg"))
    end
end

In [25]:
# MYCELIA_METADATA = joinpath(Pkg.dir("Mycelia"), "metadata")
MYCELIA_METADATA = joinpath(dirname(dirname(pathof(Mycelia))),  "metadata")

"/home/jupyter-cjprybol/.julia/dev/Mycelia/metadata"

In [26]:
# https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=Bacteriophage,%20all%20taxids&Completeness_s=complete
entity_metadata = DataFrames.DataFrame(uCSV.read("$(MYCELIA_METADATA)/2021-11-13-ncbi-complete-bacteriophage.csv", header=1, quotes='"')...)
entity_metadata = entity_metadata[entity_metadata[!, "Nuc_Completeness"] .== "complete", :]
entity_metadata = entity_metadata[entity_metadata[!, "Sequence_Type"] .== "RefSeq", :]
sort!(entity_metadata, "Accession")

,Accession,SRA_Accession,Submitters
,String,String,String
1,NC_000866.4,,"Miller,E.S., Kutter,E., Mosig,G., Arisaka,F., Kunisawa,T., Ruger,W., Ho,C.K., Shuman,S., Shcherbakov,V., Granovsky,I., Plugina,L., Shcherbakova,T., Sizova,S., Pyatkov,K., Shlyapnikov,M., Shubina,O., Xu,W., Gauss,P., Shen,J., Dunn,C.A., Bessman,M.J., Truncaite,L., Zajanckauskaite,A., Nivinskas,R., Belle,A., Landthaler,M., Shub,D.A., Stehr,M., Schneider,G., Aslund,F., Holmgren,A., Lindqvist,Y., Ramanculov,E., Young,R., Vaiskunaite,R., Miller,A., Davenport,L., Colowick,N.E., Pietz,B.C., Franklin,J.L., Haseltine,D., Wang,F.J., Ripley,L.S., Paddison,P., Abedon,S.T., Dressman,H.K., Gailbreath,K., Tracy,J., Mosser,E., Neitzel,J., Guttman,B., Kim,B.C., Kim,K., Park,E.H., Lim,C.J., Kadyrov,F.A., Shlyapnikov,M.G., Kryukov,V.M., Malys,N., Carles-Kinch,K., George,J.W., Kreuzer,K.N., Kai,T., Selick,H.E., Yonesaki,T., Penner,M., Morad,I., Snyder,L., Kaufmann,G., Bova,R., Cascino,A., Cipollaro,M., Gargano,S., Grau,O., Micheli,M.R., Santoro,M., Scarlato,V., Storlazzi,A., Young,P., Ohman,M., Sjoberg,B.M., Koch,T., Bouet,J.Y., Woszczyk,J., Repoila,F., Francois,V., Louarn,J.M., Krisch,H.M., Stormo,G.D., Dyson,R.L., Alberts,B.M., Orsini,G., Ouhammouch,M., Le Caer,J.P., Brody,E.N., Hacker,K.J., Sanson,B., Uzan,M., Sharma,M., Ellis,R.L., Hinton,D.M., Raudonikiene,A., Viteniene,I., Gruidl,M.E., Chen,T.C., Efimov,V.P., Prilipov,A.G., Mesyanzhinov,V.V., Brody,E., Favre,R., Kaliman,A.V., Khasanova,M.A., Tanyashin,V.I., Bayev,A.A., Powell,D., Franklin,J., Daegelen,P., Frazier,M.W., Maley,G.F., Duceman,B.W., Wang,A.M., Martinez,J., Maley,F., Marusich,E.I., Lin,G.W., Fan,W.H., Hahn,S., Hilse,D., Lu,M.J., Henning,U., Lamm,N., Selivanov,N.A., Black,L.W., Nikolaeva,L.I., Tseng,M.J., Hilfinger,J.M., Walsh,A., Greenberg,G.R., Spicer,E.K., Rush,J., Fung,C., Reha-Krantz,L.J., Karam,J.D., Konigsberg,W.H., Ishimoto,L.K., Ishimoto,K.S., Eiserling,F.A., Nakako,T., Takahashi,H., Ishii,S., Wang,Y., Mathews,C.K., Ishimoto,L., Kassavetis,G., Kumazaki,T., Huang,W.M., Ao,S.Z., Casjens,S., Orlandi,R., Zeikus,R., Weiss,R., Winge,D., Fang,M., Montag,D., Degen,M., Riede,I., Eschbach,M.L., Tomaschewski,J., Hsu,T., Wei,R.X., Dawson,M., Gayle,M., Winter,R.B., Gold,L., Chu,F.K., Valerie,K., Stevens,J., Lynch,M., Henderson,E.E., de Riel,J.K., Hahne,S., Mathews,C.Z., Rand,K.N., Gait,M.J., West,D.K., Belfort,M., Keller,B., Bickle,T.A., Gram,H., Crabb,J.W., Fujisawa,H., Minagawa,T., Broida,J., Abelson,J., Midgley,C.A., Murray,N.E., Parker,M.L., Christensen,A.C., Boosman,A., Stockard,J., Young,E.T., Doermann,A.H., Macdonald,P.M., Sengstag,C., Kellenberger,E., Drivdahl,R., Rand,K., Trojanowska,M., Karam,J., Stormo,G., Purohit,S., Armstrong,J., Brown,R.S., Tsugita,A., Owen,J.E., Schultz,D.W., Taylor,A., Smith,G.R., Volker,T.A., Gafner,J., Showe,M.K., Allet,B., Oliver,D.B., Crowther,R.A., Pribnow,D., Sigurdson,D.C., Singer,B.S., Napoli,C., Brosius,J., Dull,T.J., Noller,H.F., Anderson,B., Zurabishvili,T., Marusich,E., Schneider,M., Mullins,T., Napuli,A., Mesyanzhinov,V., Drake,J.W., Nguyen,D., Dressman,H., Goldberg,E.B., Ueno,H., Otsuko,Y., Morimoto,W., Kreuzer,K., Stidham,T., Thomas,E., Mzhavia,N., Djavachishvili,T., Peterson,S., Eidemiller,J., Awaya,M., Canada,W., Dimitroff,B., Blattner,F., Nakanishi,M., Alberts,B., Yasuda,G., Parker,M., Doermann,G."
2,NC_000867.1,,"Kivela,H.M., Mannisto,R.H., Kalkkinen,N., Bamford,D.H., Paulin,L., Bamford,J.K., Bamford,J.K.H."
3,NC_000871.1,,"Desiere,F., Lucchini,S., Brussow,H."
4,NC_000872.1,,"Desiere,F., Lucchini,S., Brussow,H., Bruttin,A."
5,NC_000896.1,,"Altermann,E., Klein,J.R., Henrich,B., Engel,G., Klein,J., Binishofer,B., Blasi,U., Fremaux,C., De Antoni,G.L., Raya,R.R., Klaenhammer,T.R."
6,NC_000902.1,,"Miyamoto,H., Nakai,W., Yajima,N., Fujibayashi,A., Higuchi,T., Sato,K., Matsushiro,A."
7,NC_000924.1,,"Plunkett,G. III, Rose,D.J., Durfee,T.J., Blattner,F.R."
8,NC_000929.1,,"Morgan,G.J., Hatfull,G.F., Casjens,S., Hendrix,R.W., Morgan,G., Hatfull,G., Hendrix,R."
9,NC_000935

In [27]:
Random.seed!(0)
n_samples = 10
subset_indices = StatsBase.sample(1:size(entity_metadata, 1), n_samples, ordered=true, replace=false)
entity_metadata = entity_metadata[subset_indices, :]
# n_samples = size(entity_metadata, 1)

,Accession,SRA_Accession,Submitters
,String,String,String
1,NC_005355.1,,"Ventura,M., Canchaya,C., Pridmore,R.D., Brussow,H., Canchaya,C.A., Pridmore,D., Bruessow,H."
2,NC_020844.1,,"Henn,M.R., Dillon,J., Levin,J., Malboeuf,C., Casali,M., Russ,C., Lennon,N., Chapman,S.B., Erlich,R., Young,S.K., Yandava,C., Zeng,Q., Alvarado,L., Anderson,S., Berlin,A., Chen,Z., Freedman,E., Gellesch,M., Goldberg,J., Green,L., Griggs,A., Gujja,S., Heilman,E.R., Heiman,D., Hollinger,A., Howarth,C., Larson,L., Mehta,T., Pearson,M., Roberts,A., Ryan,E., Saif,S., Shea,T., Shenoy,N., Sisk,P., Stolte,C., Sykes,S., White,J., Haas,B., Nusbaum,C., Birren,B."
3,NC_022057.1,,"Farina,J., Richards,K., Hiller,J., Gannon,D., Reifsnyder,R., Vogel,A., Ganski,A., Massaley,M., Carzo,S., Brower,C., Semler,R., Smith,V., Friel,S., Flynn,L., Moran,D.J., Wodarski,D.M., Harrison,M.A., Dunbar,D.A., Wang,X., Crowell,R., Bostrom,M.A., Burke,M., Wright,G.M., Gregory,S.G., Colman,S.D., Bradley,K.W., Khaja,R., Lewis,M.F., Barker,L.P., Asai,D.J., Bowman,C.A., Russell,D.A., Pope,W.H., Jacobs-Sera,D., Hendrix,R.W., Hatfull,G.F."
4,NC_027365.1,,"Hatfull,G.F., Jamborcic,A., Chen,D., Mazahreh,R., Pirahanchi,Y., Mcburney-Lin,J., Beckham,A., Rich,B., Cidambi,V., Brooks,G., Jones,S., Ricci-Tam,C., Evangelista,N., Shi,E., Matinrad,H., Coker,J., Hill,R., Lam,D., Zhang,J., Dimopoulos,C., Khan,S., Wong,A., Chou,J., Hwang,V., Chen,A., Zhang,C., Wu,K., Nguyen,E., Magarian,J., Nonejuie,P., Pogliano,K., Pogliano,J., Lee,V., Hendricks,S.L., Voegtly,L.J., Wang,Y., Glascock,A.L., Anderson,J., Williamson,S.M., Walstead,R.N., Carvalho,M.R.C., Johnson,A., Buck,G.A., Bradley,K.W., Khaja,R., Lewis,M.F., Barker,L.P., Jordan,T.C., Russell,D.A., Pope,W.H., Jacobs-Sera,D., Hendrix,R.W."
5,NC_028667.1,,"Pourcel,C., Midoux,C., Bourkaltseva,M., Pleteneva,E., Krylov,V."
6,NC_028974.1,,"Djamen,P.Y., Nguyen,L., Gibbs,Z.A., Donegan-Quick,R., Visi,D.K., Allen,M.S., Hughes,L.E., Bradley,K.W., Asai,D.J., Bowman,C.A., Russell,D.A., Pope,W.H., Jacobs-Sera,D., Hendrix,R.W., Hatfull,G.F."
7,NC_031230.1,,"Bandyopadhyay,A., Carlton,M.L., Kane,M.T., Panchal,N.J., Pham,Y.C., Reynolds,Z.J., Sapienza,M.S., German,B.A., McDonnell,J.E., Schafer,C.E., Yu,V.J., Warner,M.H., Furbee,E.C., Grubb,S.R., Montgomery,M.T., Garlena,R.A., Russell,D.A., Pope,W.H., Jacobs-Sera,D., Hendrix,R.W., Hatfull,G.F."
8,NC_047712.1,,"Gregory,A.C., LaButti,K., Copeland,A., Woyke,T., Sullivan,M.B."
9,NC_051670.1,,"Alvarez,R.V., Connors,B., Jenkins,J., Bandura,A.J., Lazo,A.N., Lorang,A.C., Mazariego,B.E., Mendez,P.D., Myrthil,G.D., Wright,J., Gurney,S.M.R., Garlena,R.A., Russell,D.A., Pope,W.H., Jacobs-Sera,D., Hatfull,G.F."


In [28]:
# download genomic fasta files
show(entity_metadata, allcols=true)

10×23 DataFrame
 Row │ Accession    SRA_Accession  Submitters                         Release_Date          Isolate    Species                            Genus           Family        Molecule_type  Length  Sequence_Type  Nuc_Completeness  Genotype  Segment  Publications  Geo_Location                      Country  USA     Host                               Isolation_Source  Collection_Date  BioSample  GenBank_Title                     
     │ String       String         String                             String                String     String                             String          String        String         Int64   String         String            String    String   String        String                            String   String  String                             String            String           String     String                            
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [29]:
# entity_metadata[1, "Species"]
# entity_metadata[1, "Accession"]

entity_index = 1

accession = entity_metadata[entity_index, "Accession"]
entity_genbank = BioFetch.fetchseq(accession, format = BioFetch.gb)

fasta_dna_sequences = [
    FASTX.FASTA.Record(chromosome.name, chromosome.sequence) 
        for chromosome in entity_genbank]

fasta_proteins = 
[
    FASTX.FASTA.Record(
            getproperty(gene, :protein_id),
            getproperty(gene, :product),
            GenomicAnnotations.sequence(gene; translate = true))
    for gene in GenomicAnnotations.@genes(entity_genbank, CDS)
    ]

# fasta_proteins = 
# [
#     [FASTX.FASTA.Record(
#                 getproperty(gene, :protein_id),
#                 getproperty(gene, :product),
#                 GenomicAnnotations.sequence(gene; translate = true))
#         for gene in GenomicAnnotations.@genes(chromosome, CDS)]
#             for chromosome in entity_genbank
#     ]

46-element Vector{FASTX.FASTA.Record}:
 FASTX.FASTA.Record:
   identifier: NP_958556.1
  description: site-specific integrase
     sequence: MPKRKNTSIKDYKLKSGKKRYEFVISLGQNSNGDRVQVH…
 FASTX.FASTA.Record:
   identifier: NP_958557.1
  description: Ltp family lipoprotein
     sequence: MSKKITGEDGKVYVEQKPFYKRAWFWIVVVVLVVVIGGA…
 FASTX.FASTA.Record:
   identifier: NP_958558.1
  description: hypothetical protein
     sequence: MRVFLLEIYCMPLFIPKYFIVHNANDIKLKANHGGLKVN…
 FASTX.FASTA.Record:
   identifier: NP_958559.1
  description: DUF1828 domain-containing protein
     sequence: MTLADKLLTDYVNWYKKEAKFKDLSQNVIRIEVPFLDSF…
 FASTX.FASTA.Record:
   identifier: NP_958560.1
  description: hypothetical protein
     sequence: MDLEKLSDNEVDELIDEAKHFFESFTLNVPLGKFNEDHI…
 FASTX.FASTA.Record:
   identifier: NP_958561.1
  description: ImmA/IrrE family metallo-endopeptidase
     sequence: MYQTPEQKTAARNQVIRWLMNYALDHQYGVTFSYAAEND…
 FASTX.FASTA.Record:
   identifier: NP_958562.1
  description: helix-turn-helix trans

In [30]:
k = 5

n_possible_kmers = Int(length(Mycelia.DNA_ALPHABET)^5//2)

AA_ALPHABET = collect(filter(x -> x != BioSequences.AA_Term, Mycelia.AA_ALPHABET))
DNA_ALPHABET = Mycelia.DNA_ALPHABET

(DNA_A, DNA_C, DNA_G, DNA_T)

In [38]:
function generate_all_possible_kmers(k, alphabet)
    kmer_iterator = Iterators.product([alphabet for i in 1:k]...)
    kmer_vectors = collect.(vec(collect(kmer_iterator)))
    if eltype(alphabet) == BioSymbols.AminoAcid
        kmers = BioSequences.LongAminoAcidSeq.(kmer_vectors)
    elseif eltype(alphabet) == BioSymbols.DNA
        kmers = BioSequences.LongDNASeq.(kmer_vectors)
    else
        error()
    end
    return sort!(kmers)
end

function generate_all_possible_canonical_kmers(k, alphabet)
    kmers = generate_all_possible_kmers(k, alphabet)
    if eltype(alphabet) == BioSymbols.AminoAcid
        return kmers
    elseif eltype(alphabet) == BioSymbols.DNA
        return unique!(BioSequences.canonical.(kmers))
    else
        error()
    end
end

generate_all_possible_canonical_kmers (generic function with 1 method)

In [39]:
canonical_kmers = generate_all_possible_canonical_kmers(2, AA_ALPHABET)
# canonical_kmers = generate_all_possible_canonical_kmers(2, Mycelia.DNA_ALPHABET)

484-element Vector{BioSequences.LongAminoAcidSeq}:
 AA
 AR
 AN
 AD
 AC
 AQ
 AE
 AG
 AH
 AI
 AL
 AK
 AM
 ⋮
 UL
 UK
 UM
 UF
 UP
 US
 UT
 UW
 UY
 UV
 UO
 UU

In [40]:
canonical_kmers

484-element Vector{BioSequences.LongAminoAcidSeq}:
 AA
 AR
 AN
 AD
 AC
 AQ
 AE
 AG
 AH
 AI
 AL
 AK
 AM
 ⋮
 UL
 UK
 UM
 UF
 UP
 US
 UT
 UW
 UY
 UV
 UO
 UU

In [ ]:
@assert length(canonical_kmers) == n_possible_kmers

In [ ]:
kmer_counts_matrix = zeros(n_possible_kmers, n_samples)

In [ ]:
entity_kmer_counts = Mycelia.count_canonical_kmers(BioSequences.DNAMer{5}, fasta_dna_sequences)

In [ ]:
entity_kmer_counts

In [ ]:
entity_index = 1
for (i, kmer) in enumerate(canonical_kmers)
#     @show kmer
#     @show entity_kmer_counts[kmer]
#     @show
    kmer_count = get(entity_kmer_counts, kmer, 0)
    kmer_counts_matrix[i, entity_index] = kmer_count
end

In [ ]:
kmer_counts_matrix

In [ ]:
for all samples, create a 

In [ ]:
# download protien fasta files

In [ ]:
# # determine optimal kmer-length

# # optimal_kmer_length = Int(ceil(log(4, sum(reference_phage_metadata[!, "Length"]))))
# # k = Primes.nextprime(optimal_kmer_length)
# k = 5

# kmer_counts_directory = "$PANGENOMICS/ncbi-reference-viruses/kmer-counts/$k"
# # kmer_counts_directory = "$DIR/kmer-counts/$k"
# if !isdir(kmer_counts_directory)
#     mkpath(kmer_counts_directory)
# end
# ProgressMeter.@showprogress for accession in reference_phage_metadata[!, "Accession"]
#     fasta_file =  "$PANGENOMICS/ncbi-reference-viruses/fasta/$(accession).fasta"
#     @assert isfile(fasta_file)
#     kmer_file = "$kmer_counts_directory/$(accession).$(k).counts"    
#     if !isfile(kmer_file)
#         LocusJulia.jellyfish_count(fasta=fasta_file, k=k, directory="$kmer_counts_directory")
#     end
#     @assert isfile(kmer_file)
# end

In [ ]:
# all(replace.(basename.(kmer_count_files), ".5.counts" => "") .== reference_phage_metadata[!, "Accession"])

## Generate kmer counts

In [ ]:
# kmer_count_files = ["$kmer_counts_directory/$(accession).$(k).counts" for accession in reference_phage_metadata[!, "Accession"]]

In [ ]:
# kmer_count_files = filter(f -> occursin("$k.counts", f), readdir(kmer_counts_directory, join=true))
# kmer_count_files = filter(x -> occursin(Regex("\\.$k\\.counts\$"), x) && !occursin('-', basename(x)), readdir("$(DIR)/kmer-counts", join=true))

In [ ]:
# compute the sha-256 of all of the contents of all of the kmer counts and use that to tag the kmer list and the joint kmer count directory

In [ ]:
# kmer_count_hash = hash(kmer_count_files)

In [ ]:
# joint_kmer_count_file = "$(DIR)/$k.$(kmer_count_hash).counts"
# joint_kmer_file = "$(DIR)/$k.$(kmer_count_hash).kmers"

In [ ]:
# if k <= 31
#     KMER_TYPE = BioSequences.DNAMer{k}
# else
#     KMER_TYPE = BioSequences.BigDNAMer{k}
# end

In [ ]:
# if !isfile(joint_kmer_file)
#     kmer_set = Set{KMER_TYPE}()
#     ProgressMeter.@showprogress for kmer_count_file in kmer_count_files
#         for line in eachline(kmer_count_file)
#             kmer = KMER_TYPE(first(split(line)))
#             push!(kmer_set, kmer)
#         end
#     end
#     kmer_list = sort!(collect(kmer_set)); kmer_set = nothing;

#     open(joint_kmer_file, "w") do io
#         for kmer in kmer_list
#             println(io, kmer)
#         end
#     end
# else
#     n_kmers = countlines(joint_kmer_file)
#     kmer_list = Vector{KMER_TYPE}(undef, n_kmers)
#     open(joint_kmer_file) do io
#         for (i, line) in enumerate(eachline(io))
#             kmer_list[i] = KMER_TYPE(line)
#         end
#     end
# end

In [ ]:
# n_kmers = length(kmer_list)

In [ ]:
# table_size_in_gb = round(n_kmers * n_samples * 64 / (8e+9), digits=3)

In [ ]:
# println("dense matrix would take up\n$(table_size_in_gb) Gb")

In [ ]:
# # kmer_counts_matrix = SparseArrays.spzeros(n_kmers, n_samples)
# kmer_counts_matrix = zeros(n_kmers, n_samples)

In [ ]:
# # 33 minutes to build -> should write this out to save and store to minize re-load time
# #  Progress: 100%|█████████████████████████████████████████| Time: 0:25:51
# ProgressMeter.@showprogress for (file_index, kmer_count_file) in enumerate(kmer_count_files)
#     for line in eachline(kmer_count_file)
#         kmer, kmer_count = split(line)
#         kmer_index = searchsortedfirst(kmer_list, KMER_TYPE(kmer))
#         kmer_counts_matrix[kmer_index, file_index] = Float64(parse(Int, kmer_count))
#     end
# end

In [ ]:
# kmer_counts_matrix

In [ ]:
# kmer_probability_matrix = reduce(hcat, [col ./ sum(col) for col in eachcol(kmer_counts_matrix)])

In [ ]:
# density = SparseArrays.nnz(kmer_counts_matrix) / reduce(*, size(kmer_counts_matrix))
# sparsity = count(x -> x == 0, kmer_counts_matrix) / reduce(*, size(kmer_counts_matrix))
# density = count(x -> x != 0, kmer_counts_matrix) / reduce(*, size(kmer_counts_matrix))

In [ ]:
# required for sparse arrays, but awful for dense arrays
# 1658.679602 seconds (4.44 G allocations: 82.863 GiB, 1.33% gc time)
# Kernel PCA(indim = 512, outdim = 512)
# @time pca_model = MultivariateStats.fit(MultivariateStats.KernelPCA, kmer_counts_matrix)

In [ ]:
# # Progress: 100%|█████████████████████████████████████████| Time: 0:06:23
# # Progress: 100%|█████████████████████████████████████████| Time: 0:07:19 for cosine
# # distance_matrix_raw_kmer_counts_cosine = zeros(Float64, n_samples, n_samples)
# # distance_matrix_raw_kmer_counts_euclidean = zeros(Float64, n_samples, n_samples)
# distance_matrix_raw_kmer_probability_js_divergence = zeros(Float64, n_samples, n_samples)
# ProgressMeter.@showprogress for i in 1:n_samples
#     x_counts = view(kmer_counts_matrix, :, i)
#     for j in i+1:n_samples
#         y_counts = view(kmer_counts_matrix, :, j)
#         distance_matrix_raw_kmer_probability_js_divergence[i, j] = distance_matrix_raw_kmer_probability_js_divergence[j, i] = Distances.js_divergence(x_counts, y_counts)
# #         distance_matrix_raw_kmer_counts_cosine[i, j] = distance_matrix_raw_kmer_counts_cosine[j, i] = Distances.cosine_dist(x_counts, y_counts)
# #         distance_matrix_raw_kmer_counts_euclidean[i, j] = distance_matrix_raw_kmer_counts_euclidean[j, i] = Distances.euclidean(x_counts, y_counts)
#     end
# end

In [ ]:
# @time pca_model = MultivariateStats.fit(MultivariateStats.PCA, kmer_counts_matrix)

In [ ]:
# transformed_kmer_counts_matrix = MultivariateStats.transform(pca_model, kmer_counts_matrix)

In [ ]:
# distance_matrix_transformed_kmer_counts = zeros(Float64, n_samples, n_samples)
# ProgressMeter.@showprogress for i in 1:n_samples
#     x_counts = view(transformed_kmer_counts_matrix, :, i)
#     for j in i+1:n_samples
#         y_counts = view(transformed_kmer_counts_matrix, :, j)
#         counts_distance = Distances.cosine_dist(x_counts, y_counts)
#         distance_matrix_transformed_kmer_counts[i, j] = distance_matrix_transformed_kmer_counts[j, i] = counts_distance
#     end
# end

In [ ]:
fasta_files = ["$PANGENOMICS/ncbi-reference-viruses/fasta/$(accession).fasta" for accession in reference_phage_metadata[!, "Accession"]]
fasta_list_file = "$(DIR)/fasta-list.txt"
open(fasta_list_file, "w") do io
    for f in fasta_files
        println(io, f)
    end
end
fasta_list_hash = hash(read(fasta_list_file))
mv(fasta_list_file, "$(DIR)/fasta-list.$(fasta_list_hash).txt")
fasta_list_file = "$(DIR)/fasta-list.$(fasta_list_hash).txt"

In [ ]:
fasta_map_index = Dict(fasta_file => i for (i, fasta_file) in enumerate(fasta_files))

# Mash distance matrix

In [ ]:
# s = 1_000
# # k = 5
# # 1712.918182 seconds (203.63 k allocations: 10.841 MiB, 0.01% compilation time)
# # 1244.423793 seconds (109.17 k allocations: 5.641 MiB, 0.01% compilation time)

# # 9414.157635 seconds (330.58 k allocations: 9.674 MiB, 0.00% compilation time)
# # k = 7
# # k=9
# # k = 13
# k=17

# # fasta_list_hash = hash(read(fasta_list_file))
# # run(`conda install -c bioconda mash`)
# mash_outfile = "$(DIR)/k$(k).s$(s).$(fasta_list_hash)"
# if !isfile("$(mash_outfile).msh")
#     #  62.588383 seconds (2.04 k allocations: 55.750 KiB) single thread
#     # 12.651958 seconds (18.48 k allocations: 1.131 MiB, 0.19% compilation time) 8 cores
#     @time run(pipeline(`mash sketch -p $(Sys.CPU_THREADS) -k $(k) -s $(s) -o $(mash_outfile) -l $(fasta_list_file)`, stderr="$(mash_outfile).stderr", stdout="$(mash_outfile).stdout"))
# end
# distance_file = "$(mash_outfile).dist"
# if !isfile(distance_file)
#     # 1601.517323 seconds (108.43 k allocations: 4.811 MiB)
#     # 20.337346 seconds (45.41 k allocations: 2.707 MiB, 0.33% compilation time) single thread
#     # 8.109837 seconds (17.63 k allocations: 1.095 MiB, 0.26% compilation time) 8 cores
#     @time run(pipeline(`mash dist -p $(Sys.CPU_THREADS) $(mash_outfile).msh $(mash_outfile).msh`, "$(mash_outfile).dist"))
# end

The mash dist file is very large, and I'm wondering whether there is any io write out limits.

Could we be more efficient by just streaming the output directly into a distance matrix?

We can definitely be more data storage efficient

In [ ]:
# @assert issorted(fasta_files)

In [ ]:
# mash_distance_matrix = zeros(length(fasta_files), length(fasta_files))

In [ ]:
# # 2439.550913 seconds (10.00 G allocations: 556.675 GiB, 6.38% gc time, 0.00% compilation time)
# @time for line in eachline("$(mash_outfile).dist")
#     fasta_1, fasta_2, distance_string, p_value_string, n_hits = split(line)
#     distance = parse(Float64, distance_string)
#     fasta_1_index = fasta_map_index[fasta_1]
#     fasta_2_index = fasta_map_index[fasta_2]
# #     fasta_1_index = searchsortedfirst(fasta_files, fasta_1)
# #     fasta_2_index = searchsortedfirst(fasta_files, fasta_2)
#     mash_distance_matrix[fasta_1_index, fasta_2_index] = distance
# end

In [ ]:
# mash_distance_matrix = nothing
# GC.gc()

# ANI distance matrix

In [ ]:
# run(`conda create -n fastANI -c bioconda fastANI`)

In [ ]:
# time output got truncated, but not too crazyy!
# again output got truncated, should have this write stderr to a file to not overdue output

# k = 16 is default
# k = 13 # still not finding similarities between species, genera, families
# k = 11

# k size drives runtime more than min_fraction overlap

# 620.518002 seconds (19.25 k allocations: 566.922 KiB)
# k = 9

# 18434.583602 seconds (434.14 k allocations: 14.115 MiB, 0.00% compilation time)
k = 8

# k = 7 # this ran for SOOOOO LONG. Too long

# ~ 180 seconds per run
# min fraction that must be shared to trust calculation
# min_fraction = 0.2 # default
# min_fraction = 0.5 # I think this made it worse
# min_fraction = 0.1 # didn't really see a change
# min_fraction = 0.05 # this is getting better!
# still 178 seconds
# min_fraction = 0.01

min_fraction = 0.01 # this is getting better!

fasta_ani_out = "$(DIR)/fastANI.k$(k).min_fraction-$(min_fraction).$(fasta_list_hash).out"
if !isfile(fasta_ani_out)
    @time run(pipeline(`fastANI -t $(Sys.CPU_THREADS) --minFraction $(min_fraction) --kmer $(k) --ql $(fasta_list_file) --rl $(fasta_list_file) -o $(fasta_ani_out)`, stderr="$(fasta_ani_out).stderr", stdout="$(fasta_ani_out).stdout"))
end

In [ ]:
# fastANI doesn't return values for any ANI ~< 80%, so we'll set all of those to distance 1 by default 
ani_distance_matrix = ones(n_samples, n_samples)

# OUTPUT_FILE will contain tab delimited row(s) with:
# - query genome
# - reference genome
# - ANI value
# - count of bidirectional fragment mappings
# - total query fragments
for line in eachline(fasta_ani_out)
    query_fasta, reference_fasta, average_nucleotide_identity, fragment_mapping_count, total_query_fragments = split(line)
#     query_fasta_index = searchsortedfirst(fasta_files, query_fasta)
    query_fasta_index = fasta_map_index[query_fasta]
#     reference_fasta_index = searchsortedfirst(fasta_files, reference_fasta)
    reference_fasta_index = fasta_map_index[reference_fasta]
    ani = parse(Float64, average_nucleotide_identity)
    distance = (100.0 - ani) / 100
    ani_distance_matrix[query_fasta_index, reference_fasta_index] = distance
end

In [ ]:
ani_distance_matrix

# Compare the following
- same species vs different species
- same genus vs different genus
- same family vs different family

for each of the following measures:
- raw kmer counts
- transformed kmer counts
- ANI
- mash distance

In [ ]:
# Species - species doesn't seem useful at all
# Genus - this is also heavily overlapping between inter and intra
# Family - 

In [ ]:
# show(reference_phage_metadata[1:3, :], allcols=true)

In [ ]:
unique_species = filter(!isempty, sort(unique(reference_phage_metadata[!, "Species"])))
unique_genera = filter(!isempty, sort(unique(reference_phage_metadata[!, "Genus"])))
unique_families = filter(!isempty, sort(unique(reference_phage_metadata[!, "Family"])))

Euclidean seems better than cosine distance
Is jensen shannon divergence better than euclidean
mash with k=13 and s=1_000 was not good
try smaller 9
try larger s

In [ ]:
# distance_matrix = distance_matrix_raw_kmer_counts_euclidean
# distance_metric = "euclidean"

# distance_matrix = distance_matrix_raw_kmer_counts_cosine
# distance_metric = "cosine"

# distance_matrix = distance_matrix_raw_kmer_probability_js_divergence
# distance_metric = "js_divergence"

# distance_matrix = mash_distance_matrix
# distance_metric = "mash"

distance_matrix = ani_distance_matrix
distance_metric = "ani"

In [ ]:
intra_species_distances = Float64[]
inter_species_distances = Float64[]
ProgressMeter.@showprogress for species in unique_species
    species_indices = findall(reference_phage_metadata[!, "Species"] .== species)
    other_species_indices = setdiff(1:size(distance_matrix, 1), species_indices)
    for index in species_indices
        other_indices = filter(i -> i != index, species_indices)
        append!(intra_species_distances, vec(distance_matrix[index, other_indices]))
        append!(inter_species_distances, vec(distance_matrix[index, other_species_indices]))
    end
end

In [ ]:
StatsBase.describe(StatsBase.sample(inter_species_distances, 10_000))

In [ ]:
StatsBase.describe(StatsBase.sample(intra_species_distances, 10_000))

In [ ]:
n_points = 1_000
p = StatsPlots.scatter(
    [
        [rand(Bool) ? 1 + rand()/3 : 1 - rand()/3 for i in 1:n_points],
        [rand(Bool) ? 2 + rand()/3 : 2 - rand()/3 for i in 1:n_points]
    ],
    [
        StatsBase.sample(intra_species_distances, n_points),
        StatsBase.sample(inter_species_distances, n_points)
    ],
    label = ["within species" "between species"],
    legend = :outertopright,
    alpha = 0.5,
    title = "$(distance_metric) distance @ k=$k"
)
p = StatsPlots.hline(p,
    [Statistics.mean(intra_species_distances) + 3*Statistics.std(intra_species_distances)],
    label="mean + 3σ"
)
StatsPlots.savefig(p, "$DIR/$(distance_metric)-distance-species-k$k.png")
StatsPlots.savefig(p, "$DIR/$(distance_metric)-distance-species-k$k.svg")
p

In [ ]:
intra_genera_distances = Float64[]
inter_genera_distances = Float64[]
ProgressMeter.@showprogress for genus in unique_genera
    genus_indices = findall(reference_phage_metadata[!, "Genus"] .== genus)
    other_genera_indices = setdiff(1:1:size(distance_matrix, 1), genus_indices)
    for index in genus_indices
        other_indices = filter(i -> i != index, genus_indices)
        append!(intra_genera_distances, vec(distance_matrix[index, other_indices]))
        append!(inter_genera_distances, vec(distance_matrix[index, other_genera_indices]))
    end
end

In [ ]:
StatsBase.describe(StatsBase.sample(inter_genera_distances, 10_000))

In [ ]:
StatsBase.describe(StatsBase.sample(intra_genera_distances, 10_000))

In [ ]:
n_points = 1_000
p = StatsPlots.scatter(
    [
        [rand(Bool) ? 1 + rand()/3 : 1 - rand()/3 for i in 1:n_points],
        [rand(Bool) ? 2 + rand()/3 : 2 - rand()/3 for i in 1:n_points]
    ],
    [
        StatsBase.sample(intra_genera_distances, n_points),
        StatsBase.sample(inter_genera_distances, n_points)
    ],
    label = ["within genera" "between genera"],
    legend = :outertopright,
    alpha = 0.5,
    title = "$(distance_metric) distance @ k=$k"
)
p = StatsPlots.hline(p,
    [Statistics.mean(intra_genera_distances) + 3*Statistics.std(intra_genera_distances)],
    label="mean + 3σ"
)
StatsPlots.savefig(p, "$DIR/$(distance_metric)-distance-genus-k$k.png")
StatsPlots.savefig(p, "$DIR/$(distance_metric)-distance-genus-k$k.svg")
p

In [ ]:
intra_family_distances = Float64[]
inter_family_distances = Float64[]
ProgressMeter.@showprogress for family in unique_families
    family_indices = findall(reference_phage_metadata[!, "Family"] .== family)
    other_family_indices = setdiff(1:1:size(distance_matrix, 1), family_indices)
    for index in family_indices
        other_indices = filter(i -> i != index, family_indices)
        append!(intra_family_distances, vec(distance_matrix[index, other_indices]))
        append!(inter_family_distances, vec(distance_matrix[index, other_family_indices]))
    end
end

In [ ]:
StatsBase.describe(StatsBase.sample(inter_family_distances, 10_000))

In [ ]:
StatsBase.describe(StatsBase.sample(intra_family_distances, 10_000))

In [ ]:
n_points = 1_000
p = StatsPlots.scatter(
    [
        [rand(Bool) ? 1 + rand()/3 : 1 - rand()/3 for i in 1:n_points],
        [rand(Bool) ? 2 + rand()/3 : 2 - rand()/3 for i in 1:n_points]
    ],
    [
        StatsBase.sample(intra_family_distances, n_points),
        StatsBase.sample(inter_family_distances, n_points)
    ],
    label = ["within families" "between families"],
    legend = :outertopright,
    alpha = 0.5,
    title = "$(distance_metric) distance @ k=$k"
)
p = StatsPlots.hline(p,
    [Statistics.mean(intra_family_distances) + 3*Statistics.std(intra_family_distances)],
    label="mean + 3σ"
)
StatsPlots.savefig(p, "$DIR/$(distance_metric)-distance-family-k$k.png")
StatsPlots.savefig(p, "$DIR/$(distance_metric)-distance-family-k$k.svg")
p

In [ ]:
# kmer_counts_matrix = nothing
# distance_matrix_raw_kmer_counts_cosine = nothing
# distance_matrix_raw_kmer_counts_euclidean = nothing
# GC.gc()